# Supply Chain Risk Classification with TabPFN

This notebook demonstrates how to use **TabPFN** for classification tasks in retail/CPG supply chain planning.

TabPFN is a foundation model for tabular data that outperforms traditional methods while being dramatically faster. It requires no hyperparameter tuning and works out-of-the-box.

**Use Cases Covered:**
1. **Supplier Delay Risk Prediction** (Binary Classification) - Predict which supplier deliveries will be delayed
2. **Material Shortage Prediction** (Multi-class Classification) - Predict material shortage risk levels

**Business Value:**
- Enable proactive supply risk mitigation
- Optimize safety stock and expediting decisions
- Improve on-time delivery performance

**Prerequisites:** Run `00_data_preparation` notebook first to set up the datasets.

**References:**
- [TabPFN Client GitHub](https://github.com/PriorLabs/tabpfn-client)
- [Prior Labs Documentation](https://docs.priorlabs.ai/)

## Compute Setup

We recommend running this notebook on **Serverless Compute** with the **Base Environment V4**.

To configure:
1. Click on the compute selector in the notebook toolbar
2. Select **Serverless**
3. Under Environment, choose **Base Environment V4**

Serverless compute provides fast startup times and automatic scaling, ideal for interactive notebook workflows.

## 1. Installation

In [ ]:
%pip install tabpfn-client scikit-learn pandas matplotlib --quiet

In [ ]:
dbutils.library.restartPython()

## 2. Authentication

TabPFN client requires authentication using an access token.

**Setting up Databricks Secrets (one-time setup):**

1. Create a secret scope using the Databricks CLI:
   ```bash
   databricks secrets create-scope tabpfn-client
   ```

2. Store your TabPFN token in the secret scope:
   ```bash
   databricks secrets put-secret tabpfn-client token
   ```

3. You can retrieve your TabPFN token on another machine by running:
   ```python
   import tabpfn_client
   token = tabpfn_client.get_access_token()
   print(token)
   ```

In [ ]:
import tabpfn_client

token = dbutils.secrets.get(scope="tabpfn-client", key="token")
tabpfn_client.set_access_token(token)

## 3. Configuration

Configure the catalog and schema where the prepared datasets are stored.

In [ ]:
# Configure catalog and schema (must match 00_data_preparation)
CATALOG = "tabpfn_databricks"
SCHEMA = "default"

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")

## 4. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

from tabpfn_client import TabPFNClassifier

## 5. Supplier Delay Risk Prediction (Binary Classification)

**Business Context:** Supply planners need to identify which incoming supplier deliveries are at risk of delay so they can:
- Expedite high-risk orders
- Adjust production schedules
- Communicate proactively with stakeholders

We'll use TabPFN to predict whether a supplier delivery will be delayed based on:
- Supplier characteristics (tier, country, reliability history)
- Order characteristics (quantity, value, lead time)
- External factors (port congestion, weather risk, peak season)

In [ ]:
# Load the Supplier Delay Risk dataset from Delta table
df_delay = spark.table("supplier_delay_risk").toPandas()

print(f"Dataset shape: {df_delay.shape}")
print(f"\nFeatures:")
print([col for col in df_delay.columns if col != 'is_delayed'])
print(f"\nTarget distribution (is_delayed):")
print(f"  On-time (0): {(df_delay['is_delayed'] == 0).sum()}")
print(f"  Delayed (1): {(df_delay['is_delayed'] == 1).sum()}")
print(f"  Delay rate: {df_delay['is_delayed'].mean():.1%}")

In [ ]:
# Prepare features - encode categorical columns
df_encoded = pd.get_dummies(df_delay, columns=['supplier_tier', 'supplier_country'], drop_first=True)

# Separate features and target
feature_cols = [col for col in df_encoded.columns if col != 'is_delayed']
X = df_encoded[feature_cols].values
y = df_delay['is_delayed'].values

print(f"Feature matrix shape: {X.shape}")
print(f"Number of encoded features: {len(feature_cols)}")

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

In [ ]:
# Initialize and train TabPFN classifier
clf = TabPFNClassifier()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])

print(f"TabPFN Supplier Delay Risk Prediction Results:")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  ROC AUC:  {roc_auc:.4f}")

In [ ]:
# Detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['On-Time', 'Delayed']))

In [ ]:
# Visualize high-risk deliveries
# Show deliveries with highest predicted delay probability
df_test = df_delay.iloc[X_train.shape[0]:].copy().reset_index(drop=True)
df_test['delay_probability'] = y_pred_proba[:, 1]
df_test['predicted_delayed'] = y_pred

print("Top 10 Highest Risk Deliveries:")
high_risk = df_test.nlargest(10, 'delay_probability')[[
    'supplier_tier', 'supplier_country', 'contracted_lead_time_days',
    'historical_otd_rate', 'port_congestion_index', 'delay_probability', 'is_delayed'
]]
display(high_risk)

## 6. Model Comparison for Supplier Delay Prediction

Let's compare TabPFN with other popular classifiers to demonstrate its competitive performance.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# Define models to compare
models = {
    "TabPFN": TabPFNClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
}

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate each model
results = {}
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=cv, scoring="roc_auc")
    results[name] = {"mean": scores.mean(), "std": scores.std()}
    print(f"{name:20s}: ROC AUC = {scores.mean():.4f} (+/- {scores.std():.4f})")

In [ ]:
# Visualize comparison
df_results = pd.DataFrame(results).T
df_results.columns = ["Mean ROC AUC", "Std"]
df_results = df_results.sort_values("Mean ROC AUC", ascending=True)

fig, ax = plt.subplots(figsize=(10, 5))
colors = ["#2ecc71" if name == "TabPFN" else "#3498db" for name in df_results.index]
bars = ax.barh(df_results.index, df_results["Mean ROC AUC"], color=colors)
ax.errorbar(df_results["Mean ROC AUC"], df_results.index, 
            xerr=df_results["Std"], fmt="none", color="black", capsize=3)
ax.set_xlabel("ROC AUC Score")
ax.set_title("Model Comparison - Supplier Delay Risk Prediction")
ax.set_xlim(0.5, 1.0)
plt.tight_layout()
plt.show()

## 7. Material Shortage Prediction (Multi-class Classification)

**Business Context:** Material planners need to identify which materials are at risk of shortage to:
- Prioritize procurement actions
- Expedite critical orders
- Adjust production schedules

We'll predict shortage risk levels:
- **0 = No Risk**: Adequate inventory coverage
- **1 = At Risk**: Monitor closely, may need action
- **2 = Critical**: Immediate action required

In [ ]:
# Load the Material Shortage dataset from Delta table
df_shortage = spark.table("material_shortage").toPandas()

print(f"Dataset shape: {df_shortage.shape}")
print(f"\nTarget distribution (shortage_risk):")
shortage_labels = {0: 'No Risk', 1: 'At Risk', 2: 'Critical'}
for val, label in shortage_labels.items():
    count = (df_shortage['shortage_risk'] == val).sum()
    print(f"  {val} ({label}): {count} ({count/len(df_shortage):.1%})")

In [ ]:
# Prepare features - encode categorical columns
df_shortage_encoded = pd.get_dummies(df_shortage, 
                                      columns=['material_type', 'criticality_class'], 
                                      drop_first=True)

# Separate features and target
shortage_feature_cols = [col for col in df_shortage_encoded.columns if col != 'shortage_risk']
X_shortage = df_shortage_encoded[shortage_feature_cols].values
y_shortage = df_shortage['shortage_risk'].values

print(f"Feature matrix shape: {X_shortage.shape}")

# Split the data
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X_shortage, y_shortage, test_size=0.3, random_state=42, stratify=y_shortage
)

In [ ]:
# Train TabPFN on multi-class problem
clf_shortage = TabPFNClassifier()
clf_shortage.fit(X_train_s, y_train_s)

# Make predictions
y_pred_shortage = clf_shortage.predict(X_test_s)
y_pred_proba_shortage = clf_shortage.predict_proba(X_test_s)

# Evaluate
accuracy_shortage = accuracy_score(y_test_s, y_pred_shortage)
print(f"Multi-class Classification Accuracy: {accuracy_shortage:.4f}")

print("\nClassification Report:")
print(classification_report(y_test_s, y_pred_shortage, 
                            target_names=['No Risk', 'At Risk', 'Critical']))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test_s, y_pred_shortage)

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm, interpolation='nearest', cmap='Blues')
ax.figure.colorbar(im, ax=ax)

classes = ['No Risk', 'At Risk', 'Critical']
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       xticklabels=classes, yticklabels=classes,
       title='Material Shortage Risk - Confusion Matrix',
       ylabel='Actual',
       xlabel='Predicted')

# Add text annotations
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

In [ ]:
# Identify critical materials requiring immediate attention
df_test_shortage = df_shortage.iloc[X_train_s.shape[0]:].copy().reset_index(drop=True)
df_test_shortage['predicted_risk'] = y_pred_shortage
df_test_shortage['critical_probability'] = y_pred_proba_shortage[:, 2]  # Probability of Critical

print("Top 10 Materials with Highest Critical Risk Probability:")
critical_materials = df_test_shortage.nlargest(10, 'critical_probability')[[
    'material_type', 'criticality_class', 'current_stock_days',
    'num_active_suppliers', 'avg_supplier_reliability', 
    'critical_probability', 'shortage_risk'
]]
display(critical_materials)

## 8. Business Impact Analysis

Let's quantify the potential business impact of using TabPFN for supply chain risk prediction.

In [ ]:
# Calculate potential business impact for supplier delay prediction
print("=" * 60)
print("BUSINESS IMPACT ANALYSIS: Supplier Delay Risk Prediction")
print("=" * 60)

# Assume average delay cost and expediting cost
avg_delay_cost = 5000  # Cost per delayed delivery (production disruption, expediting, etc.)
expedite_cost = 500    # Cost to proactively expedite a delivery

# Without ML: All delays incur full cost
total_delays_test = y_test.sum()
baseline_cost = total_delays_test * avg_delay_cost

# With ML: We expedite high-probability orders, preventing some delays
# Assume we expedite orders with >50% delay probability
high_risk_mask = y_pred_proba[:, 1] > 0.5
expedited_count = high_risk_mask.sum()
true_positives = ((y_pred == 1) & (y_test == 1)).sum()  # Correctly identified delays
false_positives = ((y_pred == 1) & (y_test == 0)).sum()  # Unnecessary expediting

# Assume expediting prevents 80% of delays
prevented_delays = int(true_positives * 0.8)
missed_delays = total_delays_test - prevented_delays

ml_cost = (missed_delays * avg_delay_cost) + (expedited_count * expedite_cost)
savings = baseline_cost - ml_cost

print(f"\nTest Set Statistics:")
print(f"  Total deliveries: {len(y_test)}")
print(f"  Actual delays: {total_delays_test}")
print(f"  Identified as high-risk: {expedited_count}")

print(f"\nCost Analysis (assuming ${avg_delay_cost:,} per delay, ${expedite_cost:,} per expedite):")
print(f"  Baseline cost (no ML): ${baseline_cost:,.0f}")
print(f"  With TabPFN prediction: ${ml_cost:,.0f}")
print(f"  Estimated savings: ${savings:,.0f} ({savings/baseline_cost:.1%})")

## Summary

In this notebook, we demonstrated:

- ✅ **Binary Classification**: Supplier delay risk prediction with ROC AUC analysis
- ✅ **Multi-class Classification**: Material shortage risk levels (No Risk, At Risk, Critical)
- ✅ **Model Comparison**: TabPFN vs. traditional ML algorithms
- ✅ **Business Impact**: Quantified potential cost savings from proactive risk management

**Key Takeaways:**
1. TabPFN provides competitive performance without hyperparameter tuning
2. Probability outputs enable risk-based decision making
3. Supply chain risk prediction can significantly reduce operational costs

**Next Steps:**
- Run `02_regression` notebook for price elasticity and promotion lift prediction
- Explore threshold optimization for different business objectives
- Integrate predictions into supply planning workflows